# 1. Import packages

In [1]:
# Get path for the py containing functions
import os
import sys
import importlib
module_path =r'C:\Users\shiwei\Documents\AnalysisTool_Chromatin'
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
# import gene selection modules
import gene_selection
importlib.reload(gene_selection)
import gene_to_loci
importlib.reload(gene_to_loci)
import gene_activity
importlib.reload(gene_activity)
import loci_1d_features
importlib.reload(loci_1d_features)

<module 'loci_1d_features' from 'C:\\Users\\shiwei\\Documents\\AnalysisTool_Chromatin\\loci_1d_features.py'>

In [3]:
%run "C:\Users\shiwei\Documents\ImageAnalysis3\required_files\Startup_py3.py"
sys.path.append(r"C:\Users\shiwei\Documents")

import ImageAnalysis3 as ia
%matplotlib notebook

from ImageAnalysis3 import *
print(os.getpid())

import h5py
from ImageAnalysis3.classes import _allowed_kwds
import ast

import pandas as pd

37148


# 2. Get all MERFISH loci (old codebook for example) with adjacent gene annotation

In [4]:
# load a pre-prepared adjacent gene dataframe for all MERFISH-loci
gene_fname = r"C:\Users\shiwei\Documents\AnalysisTool_Chromatin\resources\old_MERFISH_loci_adjacent_genes_tss.csv"

im_loci_df = pd.read_csv(gene_fname, index_col=0)

In [5]:
# Change column names to to be compatible with the gene score functions/pipelines below 
im_loci_df_new = im_loci_df[['name','id','chr','chr_order']]

dist_list = [0,10,50,100,500,1000,1500,2000]
adjacent_gene_cols = [f'adjacent_genes_{_dist}kb_tss' for _dist in dist_list]
for _col in adjacent_gene_cols:
    im_loci_df_new[_col.capitalize()]=im_loci_df[_col]

im_loci_df_new.head()

<ipython-input-5-3ad9719da169>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  im_loci_df_new[_col.capitalize()]=im_loci_df[_col]


,name,id,chr,chr_order,Adjacent_genes_0kb_tss,Adjacent_genes_10kb_tss,Adjacent_genes_50kb_tss,Adjacent_genes_100kb_tss,Adjacent_genes_500kb_tss,Adjacent_genes_1000kb_tss,Adjacent_genes_1500kb_tss,Adjacent_genes_2000kb_tss
loci_name,,,,,,,,,,,,
chr1_3740000_3760000,1:3740000-3760000,1,1,0,intergenic,intergenic,intergenic,Xkr4,Gm18956; Gm7341; Xkr4,Gm18956; Gm2053; Gm6085; Gm6101; Gm6119; Gm734...,Atp6v1h; Gm16041; Gm17101; Gm18956; Gm2053; Gm...,Atp6v1h; Gm16041; Gm17101; Gm18956; Gm2053; Gm...
chr1_6240000_6260000,1:6240000-6260000,2,1,1,intergenic,intergenic,4732440D04Rik; Rb1cc1,4732440D04Rik; Rb1cc1,4732440D04Rik; Gm19026; Gm2147; Npbwr1; Rb1cc1...,4732440D04Rik; Gm19026; Gm2147; Gm5694; Gm7182...,4732440D04Rik; Atp6v1h; Gm16041; Gm17101; Gm19...,4732440D04Rik; Atp6v1h; Gm16041; Gm17101; Gm19...
chr1_8740000_8760000,1:8740000-8760000,3,1,2,intergenic,intergenic,Gm15452,Gm15452,Gm15452,1700034P13Rik; 2610203C22Rik; Adhfe1; Gm15452;...,1700034P13Rik; 2610203C22Rik; Adhfe1; Arfgef1;...,1700034P13Rik; 2610203C22Rik; Adhfe1; Arfgef1;...
chr1_11240000_11260000,1:11240000-11260000,4,1,5,intergenic,intergenic,intergenic,intergenic,A830018L16Rik; Prex2,A830018L16Rik; Cpa6; Gm15603; Gm15604; Gm7560;...,1700034P13Rik; A830018L16Rik; Arfgef1; Cops5; ...,1700034P13Rik; 2610203C22Rik; A830018L16Rik; A...
chr1_13740000_13760000,1:13740000-13760000,5,1,6,intergenic,intergenic,Gm5523,Gm5523; Lactb2,Gm5523; Gm7593; Lactb2; Ncoa2; Tram1,Eya1; Gm5523; Gm7593; Lactb2; Msc; Ncoa2; Slco...,Eya1; Gm17644; Gm2383; Gm5523; Gm6216; Gm7593;...,Eya1; Gm10566; Gm17644; Gm2383; Gm5523; Gm6216...


In [6]:
# sort codebook by chromosome order (optional)
im_loci_df_new = loci_1d_features.sort_loci_df_by_chr_order(im_loci_df_new)
im_loci_df_new.head()

,name,id,chr,chr_order,Adjacent_genes_0kb_tss,Adjacent_genes_10kb_tss,Adjacent_genes_50kb_tss,Adjacent_genes_100kb_tss,Adjacent_genes_500kb_tss,Adjacent_genes_1000kb_tss,Adjacent_genes_1500kb_tss,Adjacent_genes_2000kb_tss
loci_name,,,,,,,,,,,,
chr1_3740000_3760000,1:3740000-3760000,1,1,0,intergenic,intergenic,intergenic,Xkr4,Gm18956; Gm7341; Xkr4,Gm18956; Gm2053; Gm6085; Gm6101; Gm6119; Gm734...,Atp6v1h; Gm16041; Gm17101; Gm18956; Gm2053; Gm...,Atp6v1h; Gm16041; Gm17101; Gm18956; Gm2053; Gm...
chr1_6240000_6260000,1:6240000-6260000,2,1,1,intergenic,intergenic,4732440D04Rik; Rb1cc1,4732440D04Rik; Rb1cc1,4732440D04Rik; Gm19026; Gm2147; Npbwr1; Rb1cc1...,4732440D04Rik; Gm19026; Gm2147; Gm5694; Gm7182...,4732440D04Rik; Atp6v1h; Gm16041; Gm17101; Gm19...,4732440D04Rik; Atp6v1h; Gm16041; Gm17101; Gm19...
chr1_8740000_8760000,1:8740000-8760000,3,1,2,intergenic,intergenic,Gm15452,Gm15452,Gm15452,1700034P13Rik; 2610203C22Rik; Adhfe1; Gm15452;...,1700034P13Rik; 2610203C22Rik; Adhfe1; Arfgef1;...,1700034P13Rik; 2610203C22Rik; Adhfe1; Arfgef1;...
chr1_9600471_9656458,1:9600471-9656458,1063,1,3,2610203C22Rik,2610203C22Rik,2610203C22Rik; Gm6161; Mybl1,1700034P13Rik; 2610203C22Rik; Adhfe1; Gm18300;...,1700034P13Rik; 2610203C22Rik; Adhfe1; Cops5; C...,1700034P13Rik; 2610203C22Rik; Adhfe1; Arfgef1;...,1700034P13Rik; 2610203C22Rik; Adhfe1; Arfgef1;...,1700034P13Rik; 2610203C22Rik; A830018L16Rik; A...
chr1_9749317_9845402,1:9749317-9845402,1064,1,4,Gm6195; Sgk3,1700034P13Rik; Gm6195; Sgk3; Vcpip1,1700034P13Rik; Gm6195; Mybl1; Sgk3; Vcpip1,1700034P13Rik; Gm6195; Mcmdc2; Mybl1; Sgk3; Sn...,1700034P13Rik; 2610203C22Rik; Adhfe1; Arfgef1;...,1700034P13Rik; 2610203C22Rik; Adhfe1; Arfgef1;...,1700034P13Rik; 2610203C22Rik; Adhfe1; Arfgef1;...,1700034P13Rik; 2610203C22Rik; A830018L16Rik; A...


# 3. Load AnnData for all gene expression information

In [7]:
# Get loaded adata that is pre-prepared from other notebook
#%store -r adata
import os
import scanpy as sc
# L drive below is the crick annData
scRNA_folder =r'L:\Shiwei\DNA_MERFISH_analysis\SMARTer_nuclei_MOp'
# load from here for saved h5ad
adata = sc.read(os.path.join(scRNA_folder,r'MOp_smart_sn_labeled_20220707.h5ad'))

In [8]:
# Use these three columns below for consistent labeling of our MERFISH data
adata.obs[['subclass_label_new','class_label_new','neuron_identity']]

,subclass_label_new,class_label_new,neuron_identity
barcode,,,
SM-GE653_S113_E1-50,Lamp5,GABA,Neuronal
SM-GE653_S114_E1-50,Lamp5,GABA,Neuronal
SM-GE653_S115_E1-50,Lamp5,GABA,Neuronal
SM-GE653_S116_E1-50,Lamp5,GABA,Neuronal
SM-GE653_S117_E1-50,Lamp5,GABA,Neuronal
...,...,...,...
SM-GE66H_S083_E1-50,L5/6 NP,Gluta,Neuronal
SM-GE66H_S084_E1-50,Vip,GABA,Neuronal
SM-GE66H_S085_E1-50,Pvalb,GABA,Neuronal


In [10]:
cellgroup_list = np.unique(adata.obs['subclass_label_new'].tolist())
cellgroup_list

array(['Astro', 'Endo', 'L2/3 IT', 'L4/5 IT', 'L5 ET', 'L5 IT', 'L5/6 NP',
       'L6 CT', 'L6 IT', 'L6b', 'Lamp5', 'Micro', 'OPC', 'Oligo', 'Pvalb',
       'Sncg', 'Sst', 'Vip'], dtype='<U7')

# 5. Loop for different bin size and save the result

In [16]:
# define save_path;

save_folder= r'\\10.245.74.158\Chromatin_NAS_5\Temp_file_transfer\AnalysisTool_Chromatin\resources\transcription_mean_counts_all_subclass_labels'

In [12]:
# get all avalibale bin sizes
gene_info_cols = [_col for _col in im_loci_df_new.columns if 'Adjacent' in _col]
gene_info_cols


['Adjacent_genes_0kb_tss',
 'Adjacent_genes_10kb_tss',
 'Adjacent_genes_50kb_tss',
 'Adjacent_genes_100kb_tss',
 'Adjacent_genes_500kb_tss',
 'Adjacent_genes_1000kb_tss',
 'Adjacent_genes_1500kb_tss',
 'Adjacent_genes_2000kb_tss']

In [14]:
marker_genes_df = im_loci_df_new.copy()

# Groupby means the hierachy-level to be used for calling cell groups; see options in the box above
# here we use class-level as example
marker_genes_df['Groupby'] ='subclass_label_new'

# this function should support a list of cell groups as well
# just add a list of cellgroups (separate by '; ') to the 'Compared_groups' col

marker_genes_df['Compared_groups'] ='; '.join(cellgroup_list)
print('; '.join(cellgroup_list))
marker_genes_df.head()


Astro; Endo; L2/3 IT; L4/5 IT; L5 ET; L5 IT; L5/6 NP; L6 CT; L6 IT; L6b; Lamp5; Micro; OPC; Oligo; Pvalb; Sncg; Sst; Vip


,name,id,chr,chr_order,Adjacent_genes_0kb_tss,Adjacent_genes_10kb_tss,Adjacent_genes_50kb_tss,Adjacent_genes_100kb_tss,Adjacent_genes_500kb_tss,Adjacent_genes_1000kb_tss,Adjacent_genes_1500kb_tss,Adjacent_genes_2000kb_tss,Groupby,Compared_groups
loci_name,,,,,,,,,,,,,,
chr1_3740000_3760000,1:3740000-3760000,1,1,0,intergenic,intergenic,intergenic,Xkr4,Gm18956; Gm7341; Xkr4,Gm18956; Gm2053; Gm6085; Gm6101; Gm6119; Gm734...,Atp6v1h; Gm16041; Gm17101; Gm18956; Gm2053; Gm...,Atp6v1h; Gm16041; Gm17101; Gm18956; Gm2053; Gm...,subclass_label_new,Astro; Endo; L2/3 IT; L4/5 IT; L5 ET; L5 IT; L...
chr1_6240000_6260000,1:6240000-6260000,2,1,1,intergenic,intergenic,4732440D04Rik; Rb1cc1,4732440D04Rik; Rb1cc1,4732440D04Rik; Gm19026; Gm2147; Npbwr1; Rb1cc1...,4732440D04Rik; Gm19026; Gm2147; Gm5694; Gm7182...,4732440D04Rik; Atp6v1h; Gm16041; Gm17101; Gm19...,4732440D04Rik; Atp6v1h; Gm16041; Gm17101; Gm19...,subclass_label_new,Astro; Endo; L2/3 IT; L4/5 IT; L5 ET; L5 IT; L...
chr1_8740000_8760000,1:8740000-8760000,3,1,2,intergenic,intergenic,Gm15452,Gm15452,Gm15452,1700034P13Rik; 2610203C22Rik; Adhfe1; Gm15452;...,1700034P13Rik; 2610203C22Rik; Adhfe1; Arfgef1;...,1700034P13Rik; 2610203C22Rik; Adhfe1; Arfgef1;...,subclass_label_new,Astro; Endo; L2/3 IT; L4/5 IT; L5 ET; L5 IT; L...
chr1_9600471_9656458,1:9600471-9656458,1063,1,3,2610203C22Rik,2610203C22Rik,2610203C22Rik; Gm6161; Mybl1,1700034P13Rik; 2610203C22Rik; Adhfe1; Gm18300;...,1700034P13Rik; 2610203C22Rik; Adhfe1; Cops5; C...,1700034P13Rik; 2610203C22Rik; Adhfe1; Arfgef1;...,1700034P13Rik; 2610203C22Rik; Adhfe1; Arfgef1;...,1700034P13Rik; 2610203C22Rik; A830018L16Rik; A...,subclass_label_new,Astro; Endo; L2/3 IT; L4/5 IT; L5 ET; L5 IT; L...
chr1_9749317_9845402,1:9749317-9845402,1064,1,4,Gm6195; Sgk3,1700034P13Rik; Gm6195; Sgk3; Vcpip1,1700034P13Rik; Gm6195; Mybl1; Sgk3; Vcpip1,1700034P13Rik; Gm6195; Mcmdc2; Mybl1; Sgk3; Sn...,1700034P13Rik; 2610203C22Rik; Adhfe1; Arfgef1;...,1700034P13Rik; 2610203C22Rik; Adhfe1; Arfgef1;...,1700034P13Rik; 2610203C22Rik; Adhfe1; Arfgef1;...,1700034P13Rik; 2610203C22Rik; A830018L16Rik; A...,subclass_label_new,Astro; Endo; L2/3 IT; L4/5 IT; L5 ET; L5 IT; L...


In [17]:
# run loop to process and save all results
import gene_activity
importlib.reload(gene_activity)
from gene_activity import gene_activity_average_gene_dataframe
from gene_activity import log_gene_activity 

for _col in gene_info_cols:

    print (f'Process for {_col}')
    # get gene activity for genes from each column above for each compared cell types
    print ('Append gene mean count for all query cell types.')
    marker_genes_df_new= gene_activity_average_gene_dataframe (marker_genes_df, 
                                        adata, 
                                        adjacent_gene_col = _col,
                                        #stat_method = 'wilcoxon',
                                        report_type = 'sum',  # summing all genes' mean count
                                        parallel=False, 
                                        num_threads = 8)
    
    
    # get the log10 for the mean count
    _bin_size = _col.split('_')[2].split('kb')[0]
    cols_to_be_log = [f'Activity_average_sum_genes_{_bin_size}kb_tss' for _g in cellgroup_list]
    print ('Append the log10 value for all gene mean count columns.')            
    for _log_col in cols_to_be_log:                  

        marker_genes_df_log = log_gene_activity (marker_genes_df_new, 
                                 gene_activity_col = _log_col,
                                 log_method = np.log10, 
                                 log1p=True,
                                )
        
        
    # save the processed dataframe
    print ('Save the result.')     
    save_fname = os.path.join(save_folder, f'MERFISH_loci_subclass_level_label_transcription_sum_{_bin_size}kb_tss.csv')
    marker_genes_df_log.to_csv(save_fname)
    
    
    

Process for Adjacent_genes_0kb_tss
Append gene mean count for all query cell types.
Looping for gene acitivity calculation:
Complete in 211.064s.
Append the log10 value for all gene mean count columns.
Save the result.
Process for Adjacent_genes_10kb_tss
Append gene mean count for all query cell types.
Looping for gene acitivity calculation:
Complete in 212.013s.
Append the log10 value for all gene mean count columns.
Save the result.
Process for Adjacent_genes_50kb_tss
Append gene mean count for all query cell types.
Looping for gene acitivity calculation:
Complete in 213.576s.
Append the log10 value for all gene mean count columns.
Save the result.
Process for Adjacent_genes_100kb_tss
Append gene mean count for all query cell types.
Looping for gene acitivity calculation:
Complete in 213.156s.
Append the log10 value for all gene mean count columns.
Save the result.
Process for Adjacent_genes_500kb_tss
Append gene mean count for all query cell types.
Looping for gene acitivity calcula

# 6. Load one dataframe to check

In [ ]:
# check 1000kb for example
_bin_size = 1000
save_fname = os.path.join(save_folder, f'MERFISH_loci_subclass_level_label_transcription_sum_{_bin_size}kb_tss.csv')
df_check = pd.read_csv(save_fname)

%matplotlib inline


fig, ax = plt.subplots(1, 1,dpi=100)

x_col = f'Log10_Activity_average_sum_genes_{_bin_size}kb_tss: {cellgroup_list[0]}'
y_col = f'Log10_Activity_average_sum_genes_{_bin_size}kb_tss: {cellgroup_list[1]}'
ax = sns.scatterplot(data=df_check , 
            x=x_col,
            y=y_col, s=5,color='gray')